In [1]:
# Importar librerías
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer

Se detectan valores incorrectos como '?' y valores negativos. Estos valores se entienden como un dato NaN.

In [2]:
data_train = pd.read_csv('train.csv')

In [3]:
columns_train = data_train.columns
for column in columns_train:
    """ Si el valor de la columna es '?' o menor a 0, se reemplaza por None. """
    data_train[column] = data_train[column].replace('?', None)
    data_train[column] = data_train[column].astype(float)
    data_train[column] = data_train[column].apply(lambda x: None if x < 0 else x)

data_train.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
0,51.0,1.0,1.0,125.0,213.0,0.0,2.0,125.0,1.0,1.4,1.0,1.0,3.0,0.0
1,54.0,1.0,3.0,120.0,237.0,0.0,0.0,150.0,1.0,1.5,NaN,NaN,7.0,2.0
2,63.0,1.0,4.0,140.0,0.0,NaN,2.0,149.0,0.0,2.0,1.0,NaN,NaN,2.0
3,52.0,0.0,2.0,140.0,NaN,0.0,0.0,140.0,0.0,0.0,NaN,NaN,NaN,0.0
4,55.0,1.0,4.0,140.0,217.0,0.0,0.0,111.0,1.0,5.6,3.0,0.0,7.0,3.0


In [4]:
data_test = pd.read_csv('test.csv')

columns_test = data_test.columns
for column in columns_test:
    """ Si el valor de la columna es '?' o menor a 0, se reemplaza por None. """
    data_test[column] = data_test[column].replace('?', None)
    data_test[column] = data_test[column].astype(float)
    data_test[column] = data_test[column].apply(lambda x: None if x < 0 else x)

data_test.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,57.0,1.0,4.0,156.0,173.0,0.0,2.0,119.0,1.0,3.0,3.0,NaN,NaN
1,52.0,1.0,2.0,160.0,196.0,0.0,0.0,165.0,0.0,0.0,NaN,NaN,NaN
2,48.0,1.0,2.0,100.0,NaN,0.0,0.0,100.0,0.0,0.0,NaN,NaN,NaN
3,62.0,1.0,4.0,115.0,0.0,NaN,0.0,128.0,1.0,2.5,3.0,NaN,NaN
4,51.0,1.0,3.0,110.0,175.0,0.0,0.0,123.0,0.0,0.6,1.0,0.0,3.0


In [5]:
data_train['age*chol'] = data_train['age'] * data_train['chol']
data_train['age*trestbps'] = data_train['age'] * data_train['trestbps']

data_test['age*chol'] = data_test['age'] * data_test['chol']
data_test['age*trestbps'] = data_test['age'] * data_test['trestbps']

data_train = data_train.drop(['chol', 'trestbps'], axis=1)
data_test = data_test.drop(['chol', 'trestbps'], axis=1)

#### 01. Completar valores NaN por KNN

In [6]:
X = data_train.drop('label', axis=1)
y = data_train['label'] 

# Inicializar el imputador KNN
knn_imputer = KNNImputer(n_neighbors=10)
X_imputed = knn_imputer.fit_transform(X)

X_imputed_df = pd.DataFrame(X_imputed, columns=X.columns)

data_train_imputed = pd.concat([X_imputed_df, y], axis=1)

data_train_imputed.head()

,age,sex,cp,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,age*chol,age*trestbps,label
0,51.0,1.0,1.0,0.0,2.0,125.0,1.0,1.4,1.0,1.0,3.0,10863.0,6375.0,0.0
1,54.0,1.0,3.0,0.0,0.0,150.0,1.0,1.5,1.6,0.7,7.0,12798.0,6480.0,2.0
2,63.0,1.0,4.0,0.3,2.0,149.0,0.0,2.0,1.0,0.7,6.1,0.0,8820.0,2.0
3,52.0,0.0,2.0,0.0,0.0,140.0,0.0,0.0,1.9,0.6,5.7,8618.9,7280.0,0.0
4,55.0,1.0,4.0,0.0,0.0,111.0,1.0,5.6,3.0,0.0,7.0,11935.0,7700.0,3.0


In [7]:
X_test = data_test

X_test_imputed = knn_imputer.transform(X_test)

data_test_imputed = pd.DataFrame(X_test_imputed, columns=X_test.columns)
data_test_imputed.head()

,age,sex,cp,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,age*chol,age*trestbps
0,57.0,1.0,4.0,0.0,2.0,119.0,1.0,3.0,3.0,0.7,5.9,9861.0,8892.0
1,52.0,1.0,2.0,0.0,0.0,165.0,0.0,0.0,1.6,0.2,5.6,10192.0,8320.0
2,48.0,1.0,2.0,0.0,0.0,100.0,0.0,0.0,1.9,0.1,5.3,9119.0,4800.0
3,62.0,1.0,4.0,0.3,0.0,128.0,1.0,2.5,3.0,0.5,5.8,0.0,7130.0
4,51.0,1.0,3.0,0.0,0.0,123.0,0.0,0.6,1.0,0.0,3.0,8925.0,5610.0


#### 02. Normalizar los valores con StandarScaler

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [9]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

In [10]:
data_train_scaled = pd.DataFrame(X_scaled, columns=X.columns)
data_train_scaled = pd.concat([data_train_scaled, y], axis=1)

In [11]:
data_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

#### 03. Modelo Random Forest

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

# GBM
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report

In [13]:
X_train = data_train_scaled.drop('label', axis=1)
y_train = data_train_scaled['label']

# Inicializar el modelo de Bosques Aleatorios
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [14]:
y_pred = rf_model.predict(data_test_scaled)

df = pd.DataFrame({'ID': range(len(y_pred)), 'label': y_pred})
df['label'] = df['label'].astype(int)
df.to_csv('result.csv', index=False)
